In [1]:
import time
import uuid
import requests
import json
!pip install -q websocket-client
import websocket


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [32]:
URL = "18.236.68.146"

In [3]:
from concurrent.futures import ThreadPoolExecutor

def run_in_parallel(func, n):
    # Ensure the callable and repetitions are valid
    if not callable(func) or not isinstance(n, int) or n < 1:
        raise ValueError("Invalid function or number of repetitions")
    
    # Define a wrapper function to execute
    def task_wrapper():
        return func()
    
    # Use ThreadPoolExecutor to run tasks in parallel
    with ThreadPoolExecutor(max_workers=n) as executor:
        futures = [executor.submit(task_wrapper) for _ in range(n)]
        
        # Wait for all futures to complete and collect results
        results = [future.result() for future in futures]
    
    return results


## Gradio 4

In [33]:
def request():
	start_time = time.time()
	session_hash = uuid.uuid4().hex
	payload = {"data": ["test"], "fn_index": 0, "session_hash": session_hash}
	url = f"http://{URL}/"
	resp = requests.post(f"{url}queue/join", json=payload, timeout=5)
	assert resp.status_code == 200

	message_count = 0
	output = ""
	with requests.get(f"{url}queue/data?session_hash={session_hash}", stream=True) as response:
		response.raise_for_status()
		for line in response.iter_lines():
			if line:
				decoded_line = line.decode('utf-8')
				if decoded_line.startswith("data:"):
					data = decoded_line.replace("data: ", "")
					if "close_stream" in data:
						break
					output = data
					message_count += 1
	
	end_time = time.time()
	duration = end_time - start_time
	return (duration, message_count, json.loads(output)["output"]["data"])

In [34]:
request()

(5.6847522258758545,
 504,
 [[['test',
    'Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lo

In [ ]:
output = run_in_parallel(request, 5)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 25)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [86]:
output = run_in_parallel(request, 100)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)

print(avg_duration, avg_msg)


83.2809367132187 535.58


In [36]:
output = run_in_parallel(request, 250)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


95.03129840755463 526.68


## Gradio 3.x

In [8]:
def request():
    start_time = time.time()
    url = f"ws://{URL}/"
    session_hash = uuid.uuid4().hex
    output = None
    message_count = 0
    start_time = time.time()
    try:
        # Connect to WebSocket server
        ws = websocket.create_connection(f"{url}queue/join")

        while True:
            message = ws.recv()  # Wait and receive incoming message
            message_count += 1
            message = json.loads(message)
            msg = message["msg"]

            if msg == "send_hash":
                ws.send(json.dumps({"session_hash": session_hash, "fn_index": 0}))

            if msg == "send_hash":
                ws.send(json.dumps({"data":["test"],"event_data":None,"fn_index":0,"session_hash":session_hash}	))

            if msg == "process_completed":
                output = message["output"]["data"]
                break


    finally:
        ws.close()  # Ensure the connection is closed properly
        
    duration = time.time() - start_time
    return duration, message_count, output


In [9]:
request()

(7.217260122299194,
 506,
 [[['test',
    'Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lor

In [ ]:
output = run_in_parallel(request, 5)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 25)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)

In [10]:
output = run_in_parallel(request, 100)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


18.25279019832611 506.0


In [11]:
output = run_in_parallel(request, 250)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


46.61669938659668 506.408


### Simple SSE

In [ ]:
def request():
	start_time = time.time()
	url = f"http://{URL}/"
	message_count = 0
	output = ""
	with requests.get(f"{url}sse", stream=True) as response:
		response.raise_for_status()
		for line in response.iter_lines():
			if line:
				decoded_line = line.decode('utf-8')
				if decoded_line.startswith("data:"):
					output = decoded_line.replace("data: ", "")
					message_count += 1
			if message_count == 500:
				break
	
	end_time = time.time()
	duration = end_time - start_time
	return (duration, message_count, output)

In [ ]:
request()

In [ ]:
output = run_in_parallel(request, 5)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 25)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 100)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 250)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


### Simple Websocket

In [ ]:
def request():
    start_time = time.time()
    url = f"ws://{URL}/"
    output = None
    message_count = 0
    start_time = time.time()
    try:
        # Connect to WebSocket server
        ws = websocket.create_connection(f"{url}ws")

        while True:
            message = ws.recv()  # Wait and receive incoming message
            message_count += 1
            output = message
            if message_count == 500:
                break


    finally:
        ws.close()  # Ensure the connection is closed properly
        
    duration = time.time() - start_time
    return duration, message_count, output


In [ ]:
request()

In [ ]:
output = run_in_parallel(request, 5)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 25)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 100)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 250)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


### SSE w/ Workers

In [98]:
def request():
	start_time = time.time()
	payload = {"data": "test"}
	url = f"http://{URL}/"
	resp = requests.post(f"{url}sse/send", json=payload, timeout=5)
	assert resp.status_code == 200
	session_id = resp.json()["session_id"]

	message_count = 0
	output = ""
	with requests.get(f"{url}sse/listen?session_id={session_id}", stream=True) as response:
		response.raise_for_status()
		for line in response.iter_lines():
			if line:
				decoded_line = line.decode('utf-8')
				if decoded_line.startswith("data:"):
					output = decoded_line.replace("data: ", "")
					message_count += 1
	
	end_time = time.time()
	duration = end_time - start_time
	return (duration, message_count, output)

In [99]:
request()

(5.302220106124878,
 500,
 'Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem 

In [ ]:
output = run_in_parallel(request, 5)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 25)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [100]:
output = run_in_parallel(request, 100)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


5.664539747238159 500.0


In [101]:
output = run_in_parallel(request, 250)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


11.787519352912902 500.0


### Websockets w/ Workers

In [104]:
def request():
    start_time = time.time()
    url = f"ws://{URL}/"
    output = None
    message_count = 0
    start_time = time.time()
    try:
        ws = websocket.create_connection(f"{url}ws")
        ws.send("test")

        while True:
            message = ws.recv()  # Wait and receive incoming message
            message_count += 1
            output = message
            if message_count == 500:
                break
        

    finally:
        ws.close()  # Ensure the connection is closed properly
        
    duration = time.time() - start_time
    return duration, message_count, output


In [105]:
request()

(5.335849285125732,
 500,
 'Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem Lorem 

In [ ]:
output = run_in_parallel(request, 5)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [ ]:
output = run_in_parallel(request, 25)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


In [106]:
output = run_in_parallel(request, 100)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


16.153259081840517 500.0


In [107]:
output = run_in_parallel(request, 250)
avg_duration = sum(o[0] for o in output) / len(output)
avg_msg = sum(o[1] for o in output) / len(output)
print(avg_duration, avg_msg)


40.5195159034729 500.0
